# This notebook demonstrates (Retrieval Augmented Generation) RAG with SQLLite database using LLamaIndex Agent framework.

## Install necessary packages

In [2]:
!pip install -q pandas
!pip install -q openai
!pip install -q llama-index-core
!pip install -q llama-index-llms-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.6 MB/s eta 0:00:00


## Import required modules

In [3]:
#from bsedata.bse import BSE
import pandas as pd
import sqlite3
import os
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import AgentWorkflow, FunctionAgent
from google.colab import userdata

## Get the OpenAI Api Key and intialize the Open AI LLM Client

In [6]:
# Get the OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
llm = OpenAI("gpt-4o-mini")

# Load the client data into pandas dataframe and peform basic inspectin

In [7]:
client_data='/content/Client_Company_List.xlsx'
df_client_data=pd.read_excel(client_data)
df_client_data.head()

,Company Name,Scrip Code,Sector
0,Life Insurance Corporation of India,543526,Life Insurance
1,Tata Consultancy Services Ltd,532540,Technology
2,Infosys Ltd,500209,Technology
3,ITC Ltd,500875,FMCG
4,LTIMindtree Ltd,540005,Technology


In [8]:
df_client_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Company Name  25 non-null     object
 1   Scrip Code    25 non-null     object
 2   Sector        25 non-null     object
dtypes: object(3)
memory usage: 732.0+ bytes


### Rename columns for easier interaction with SQLLite database

In [9]:
df_client_data.rename(columns={'Company Name':'Company_Name', 'Scrip Code' : 'Scrip_Code'},inplace=True)
df_client_data.describe()

,Company_Name,Scrip_Code,Sector
count,25,25,25
unique,25,25,9
top,Life Insurance Corporation of India,543526,Technology
freq,1,1,6


# Set up sqllite database and load the data from the dataframe

In [10]:
db_path = '/content/db_client_data'
if not os.path.exists(db_path):

    db_conn = sqlite3.connect(db_path)
    df_client_data.to_sql('client_data', db_conn, if_exists='replace', index=False)
    db_conn.close()

print('Database and tables created. Data populated in tables.')

Database and tables created. Data populated in tables.


In [11]:
tab_client_data = pd.read_sql_query('select * from client_data', sqlite3.connect(db_path))
tab_client_data.head()

,Company_Name,Scrip_Code,Sector
0,Life Insurance Corporation of India,543526,Life Insurance
1,Tata Consultancy Services Ltd,532540,Technology
2,Infosys Ltd,500209,Technology
3,ITC Ltd,500875,FMCG
4,LTIMindtree Ltd,540005,Technology


# Define an LLM agent using LLamaIndex

### Define a tool for the LLM agent to fetch the current stock prices

In [45]:
def execute_query(query: str, db_path: str)->list:
  """
      Execute the query and return the output of the query in a list

    Args:
        query (str): The SQL query
        db_path (str): Path to the database
        Returns:
        list: List containing output of the query
    """

  db_conn = sqlite3.connect(db_path)
  db_cur=db_conn.cursor()
  db_cur.execute(query)
  output_list=db_cur.fetchall()
  return output_list

# Test the function before passing it to LLM tool
#client_cnt=execute_query("select Scrip_Code from client_data where Company_Name in ('ITC Ltd','Infosys Ltd')",db_path)
#client_cnt=execute_query("<sql>select Scrip_Code from client_data where Company_Name = 'ITC Ltd'</sql>",db_path)
#print(client_cnt)

In [44]:
def get_schema_defn(db_path):
    """
      Get the schema definition of the SQLLite database
    Args:
        db_path (str): Path to the database
        Returns:
        str: Schema Definition of the SQLLite database
    """
    db_conn = sqlite3.connect(db_path)
    db_cur=db_conn.cursor()
    db_cur.execute("SELECT name from sqlite_master where type = 'table';")
    table_list=db_cur.fetchall()
    schema_def=[]
    for (table_name,) in table_list:
        db_cur.execute(f'PRAGMA table_info ({table_name})')
        columns=db_cur.fetchall()
        table_def=''.join(f'Table {table_name}:\n')
        table_def+= ''.join(f' {col[1]} ({col[2]})\n' for col in columns)
        schema_def.append(table_def)
    db_conn.close()
    return schema_def


### Define a prompt with clear instructions for the LLM to provide a relevant response

In [35]:
prompt="""You are an Assistant that converts user questions into SQLLite SQL query using tools.

You should peform the following steps:

Step 1:
Use the tool get_schema_defn to get the schema definition of the SQLLite database.

Step 2:
Based on the schema definition, convert user's question into SQLLite SQL query as per the following examples:

Example 1:
Question: Get me the stock prices of the clients from the Tyres Sector
SQL Query: "select Scrip_Code from client_data where sector = 'Tyres'"

Example 2:
Question: Get me the stock prices for ITC Ltd and Infosys Ltd
SQL Query: "select Scrip_Code from client_data where Company_Name in ('ITC Ltd','Infosys Ltd')"

Provide only the SQL query in your response, enclosed in <sql> tags.

"""

In [36]:
prompt

'You are an Assistant that converts user questions into SQLLite SQL query using tools.\n\nYou should peform the following steps:\n\nStep 1:\nUse the tool get_schema_defn to get the schema definition of the SQLLite database.\n\nStep 2:\nBased on the schema definition, convert user\'s question into SQLLite SQL query as per the following examples:\n\nExample 1:\nQuestion: Get me the stock prices of the clients from the Tyres Sector\nSQL Query: "select Scrip_Code from client_data where sector = \'Tyres\'"\n\nExample 2:\nQuestion: Get me the stock prices for ITC Ltd and Infosys Ltd\nSQL Query: "select Scrip_Code from client_data where Company_Name in (\'ITC Ltd\',\'Infosys Ltd\')"\n\nProvide only the SQL query in your response, enclosed in <sql> tags.\n\n'

In [ ]:
workflow=FunctionAgent(tools=[get_schema_defn], llm=llm, system_prompt=prompt)
response = await workflow.run(user_msg="Get me the stock prices for ITC Ltd")
print(response)

<sql>select Scrip_Code from client_data where Company_Name = 'ITC Ltd'</sql>


### Now let's add execute_query tool to the agent to get the client list. The prompt needs to be revised as well.

In [47]:
prompt=f"""You are an Assistant that converts user questions into SQLLite SQL query and returns the client list using tools.

You should peform the following steps:

Step 1:
Use the tool get_schema_defn to get the schema definition of the SQLLite database.

Step 2:
Based on the schema definition, convert user's question into SQLLite SQL query as per the following examples:

Example 1:
Question: Get me the stock prices of the clients from the Tyres Sector
SQL Query: "select Scrip_Code from client_data where sector = 'Tyres'"

Example 2:
Question: Get me the stock prices for ITC Ltd and Infosys Ltd
SQL Query: "select Scrip_Code from client_data where Company_Name in ('ITC Ltd','Infosys Ltd')"

Step 3:
Use the tool execute_query with database path {db_path} to execute the SQL query from Step 2 and get the output list.

Provide the output list only in your response. The response should be in the below format for each row:

Scrip Code 1: <output list[0]>
Scrip Code 2: <output list[1]>

"""
print(prompt)


You are an Assistant that converts user questions into SQLLite SQL query and returns the client list using tools.

You should peform the following steps:

Step 1:
Use the tool get_schema_defn to get the schema definition of the SQLLite database.

Step 2:
Based on the schema definition, convert user's question into SQLLite SQL query as per the following examples:

Example 1:
Question: Get me the stock prices of the clients from the Tyres Sector
SQL Query: "select Scrip_Code from client_data where sector = 'Tyres'"

Example 2:
Question: Get me the stock prices for ITC Ltd and Infosys Ltd
SQL Query: "select Scrip_Code from client_data where Company_Name in ('ITC Ltd','Infosys Ltd')"

Step 3:
Use the tool execute_query with database path /content/db_client_data to execute the SQL query from Step 2 and get the output list.

Provide the output list only in your response. The response should be in the below format for each row:

Scrip Code 1: <output list[0]>
Scrip Code 2: <output list[1]>




#### Call the LLM agent with the additional tool, revised prompt and validate the response.

In [48]:
workflow=FunctionAgent(tools=[get_schema_defn, execute_query], llm=llm, system_prompt=prompt)
response = await workflow.run(user_msg="What is the scrip code for ITC Ltd?")
print(response)

Scrip Code 1: 500875
